### Matrices Indicating the Number of Times a Taxon of Cyanobacteria Coexist with a Taxon of Non-cyanobacteria (Each level)

In [2]:
import os
import pandas as pd
import plotly.express as px
REPO = os.path.abspath('').removesuffix('Stats_Analyses')
dfcoexAna = pd.read_excel(REPO + 'Data/Obj4.xlsx', sheet_name='CoexAna')
CoexAna = dfcoexAna.values.tolist()
## Selected isolate with at least one cyanobacteria MAG and one co-existing MAG
## order as a try out first

def process(num):
    cyanodict = {}
    Allcoexist = []
    for b in CoexAna:
        prj = b[6].strip()
        for d in b[4].split("\n"):
            ecyntax = d.split(";")
            cyaltax = ecyntax[num].strip()
            for c in b[5].split("\n"):
                if c.strip() == "Unclassified":
                    general = c
                else:
                    ecoetax = c.split(";")
                    general = ecoetax[num]
                general = general.strip()
                if general != '' and cyaltax != '':
                    if cyaltax not in cyanodict:
                        cyanodict[cyaltax] = {}
                    # cyanodict
                    if general not in cyanodict[cyaltax]:
                        cyanodict[cyaltax][general] = [1, [prj]]
                    else:
                        cyanodict[cyaltax][general][0] += 1
                        if prj not in cyanodict[cyaltax][general][1]:
                            cyanodict[cyaltax][general][1].append(prj)
                    # Allcoexist
                    if general not in Allcoexist:
                        Allcoexist.append(general)
    return (cyanodict, Allcoexist)

# Process cyanodict for heatmap; Cyano = y, Coexist = x (Allcoexist)
def HMprocess(ntuple):
    ncyanodict = ntuple[0]
    nAllcoexist = ntuple[1]
    length = len(nAllcoexist)
    ylabel = []
    heatmapval = []
    corrprj = []
    for key in ncyanodict:
        if key not in ylabel:
            ylabel.append(key)
        splist = [0] * length
        pjlist = [0] * length
        for spkey in ncyanodict[key]:
            splist[nAllcoexist.index(spkey)] = ncyanodict[key][spkey][0]
            pjlist[nAllcoexist.index(spkey)] = len(ncyanodict[key][spkey][1])
        heatmapval.append(splist)
        corrprj.append(pjlist)
    return (ylabel, heatmapval, corrprj)

### Statistic Test to Evalute the Significance of Taxa Coexistances

In [3]:
# Statistic Test (Poisson distribution) p-val = 0.01
from scipy.stats import poisson
def calcpoisson(lofl):
    pmf = []
    for x in lofl:
        tot = 0
        numcolumns = len(x)
        for y in x:
            tot += y
        rowmu = tot/numcolumns
        rowpmf = []
        for y in x:
            if y >= rowmu:
                epmf = poisson.pmf(y, rowmu)
                rowpmf.append(epmf)
            else:
                rowpmf.append(-1)
        pmf.append(rowpmf)
    return pmf

def filterbyp(Rpmf, cyanolist, coexistlist, lofl, prjlist):
    filtered = []
    for a in range(0, len(Rpmf)):
        for b in range(0, len(Rpmf[a])):
            if Rpmf[a][b] < 0.01 and Rpmf[a][b] >= 0 and prjlist[a][b] > 1:
                sig = [cyanolist[a], coexistlist[b], lofl[a][b], Rpmf[a][b], prjlist[a][b]]
                filtered.append(sig)
    return filtered

In [4]:
# HeatMap for phylum
phylumtuple = process(1)
phylumheat = HMprocess(phylumtuple)
import plotly.graph_objects as go
figphylum = go.Figure(data=go.Heatmap(
                   z=phylumheat[1],
                   x=phylumtuple[1],
                   y=phylumheat[0],
                   hoverongaps = False))
figphylum.show()

# Stats
phylumpmf = calcpoisson(phylumheat[1])
sigphylum = filterbyp(phylumpmf, phylumheat[0], phylumtuple[1], phylumheat[1], phylumheat[2])
print(sigphylum)

[['Cyanobacteria', 'Bacteroidota', 36, 0.0007194135262133596, 21], ['Cyanobacteria', 'Proteobacteria', 147, 3.7357880745653446e-72, 27]]


In [5]:

# HeatMap for class
classtuple = process(2)
classheat = HMprocess(classtuple)
figclass = go.Figure(data=go.Heatmap(
                   z=classheat[1],
                   x=classtuple[1],
                   y=classheat[0],
                   hoverongaps = False))
figclass.show()

# Stats
classpmf = calcpoisson(classheat[1])
sigclass = filterbyp(classpmf, classheat[0], classtuple[1], classheat[1], classheat[2])
print(sigclass)

[['Cyanobacteriia', 'Bacteroidia', 35, 2.8924898891722123e-06, 20], ['Cyanobacteriia', 'Alphaproteobacteria', 126, 2.1980979370943095e-71, 26]]


In [6]:
# HeatMap for order
ordertuple = process(3)
orderheat = HMprocess(ordertuple)
figorder = go.Figure(data=go.Heatmap(
                   z=orderheat[1],
                   x=ordertuple[1],
                   y=orderheat[0],
                   hoverongaps = False))
figorder.show()

# Stats
orderpmf = calcpoisson(orderheat[1])
sigorder = filterbyp(orderpmf, orderheat[0], ordertuple[1], orderheat[1], orderheat[2])
print(sigorder)

[['PCC-6307', 'Rhizobiales', 4, 0.0036161044126699557, 3], ['PCC-6307', 'Caulobacterales', 4, 0.0036161044126699557, 3], ['Cyanobacteriales', 'Rhizobiales', 27, 1.5318014400585756e-18, 9], ['Cyanobacteriales', 'Sphingomonadales', 11, 7.67263507010461e-05, 7], ['Cyanobacteriales', 'Caulobacterales', 13, 3.4184605286914596e-06, 7], ['PCC-7336', 'Acetobacterales', 6, 0.0004553423298393011, 3], ['PCC-7336', 'SpSt-205', 5, 0.002795590118083154, 2], ['PCC-7336', 'Deinococcales', 7, 6.357052007496742e-05, 3], ['PCC-7336', 'Rubrobacterales', 5, 0.002795590118083154, 4], ['PCC-7336', 'Fimbriimonadales', 5, 0.002795590118083154, 3], ['PCC-7336', 'Bacillales', 6, 0.0004553423298393011, 6], ['Thermosynechococcales', 'Caulobacterales', 5, 0.00019283105686129575, 2], ['Phormidesmiales', 'Caulobacterales', 5, 0.0001579506926334984, 2]]


In [7]:
# HeatMap for family
familytuple = process(4)
familyheat = HMprocess(familytuple)
figfamily = go.Figure(data=go.Heatmap(
                   z=familyheat[1],
                   x=familytuple[1],
                   y=familyheat[0],
                   hoverongaps = False))
figfamily.update_layout(width=1200)
figfamily.show()

# Stats
familypmf = calcpoisson(familyheat[1])
sigfamily = filterbyp(familypmf, familyheat[0], familytuple[1], familyheat[1], familyheat[2])
print(sigfamily)

[['Cyanobiaceae', 'Flavobacteriaceae', 3, 0.009381697498746476, 3], ['Cyanobiaceae', 'Rhizobiaceae', 3, 0.009381697498746476, 2], ['Nostocaceae', 'Rhizobiaceae', 7, 3.203158783527193e-05, 4], ['Nostocaceae', 'Sphingomonadaceae', 5, 0.0017651575632939389, 4], ['Microcystaceae', 'Burkholderiaceae', 5, 0.0004556941308969356, 2], ['Microcystaceae', 'Sphingomonadaceae', 5, 0.0004556941308969356, 3], ['Microcystaceae', 'Caulobacteraceae', 4, 0.003588591280813363, 3], ['JA-3-3Ab', 'Acetobacteraceae', 6, 7.095969212121838e-05, 3], ['JA-3-3Ab', 'SpSt-205', 5, 0.0006237852005074546, 2], ['JA-3-3Ab', 'Thermaceae', 7, 6.918972247647147e-06, 3], ['JA-3-3Ab', 'Rubrobacteraceae', 5, 0.0006237852005074546, 4], ['JA-3-3Ab', 'GBS-DC', 5, 0.0006237852005074546, 3], ['JA-3-3Ab', 'Caldilineaceae', 4, 0.004569589259531351, 2], ['JA-3-3Ab', 'Anoxybacillaceae', 6, 7.095969212121838e-05, 6], ['Phormidesmiaceae', 'DSM-21159', 3, 0.005005376533710786, 2], ['Phormidesmiaceae', 'Maricaulaceae', 3, 0.00500537653371

In [8]:

# HeatMap for genus
genustuple = process(5)
genusheat = HMprocess(genustuple)
# Manually updated 65-9
genustuple[1][genustuple[1].index('65-9')] = "65-9\b"
figgenus = go.Figure(data=go.Heatmap(
                   z=genusheat[1],
                   x=genustuple[1],
                   y=genusheat[0],
                   hoverongaps = False))
figgenus.update_layout(width=1680, height=600)
figgenus.show()

# Stats
genuspmf = calcpoisson(genusheat[1])
siggenus = filterbyp(genuspmf, genusheat[0], genustuple[1], genusheat[1], genusheat[2])
print(siggenus)

[['Microcystis', 'Brevundimonas', 4, 0.0006075649711075708, 3], ['JA-3-3Ab', 'Elioraea_A', 3, 0.004777819636598799, 2], ['JA-3-3Ab', 'Meiothermus', 7, 7.885367804450248e-08, 3], ['JA-3-3Ab', 'Rubrobacter_C', 5, 2.812778284714299e-05, 4], ['JA-3-3Ab', 'GBS-DC', 5, 2.812778284714299e-05, 3], ['JA-3-3Ab', 'Caldilinea', 4, 0.00040986197862979837, 2], ['JA-3-3Ab', 'Anoxybacillus', 5, 2.812778284714299e-05, 5], ['Halomicronema', 'UBA5336', 2, 0.006152316715590145, 2], ['Halomicronema', 'Algihabitans', 2, 0.006152316715590145, 2], ['Halomicronema', 'Oceanicaulis', 3, 0.0002412673221800056, 2]]


In [9]:
# HeatMap for species
speciestuple = process(6)
speciesheat = HMprocess(speciestuple)
figspecies = go.Figure(data=go.Heatmap(
                   z=speciesheat[1],
                   x=speciestuple[1],
                   y=speciesheat[0],
                   hoverongaps = False))
figspecies.update_layout(height=700)
figspecies.show()

# Stats
speciespmf = calcpoisson(speciesheat[1])
sigspecies = filterbyp(speciespmf, speciesheat[0], speciestuple[1], speciesheat[1], speciesheat[2])
print(sigspecies)

[['JA-3-3Ab sp000013205', 'Meiothermus taiwanensis', 5, 0.0001579506926334984, 3]]
